<a href="https://colab.research.google.com/github/gitary/SMA_twitter/blob/main/SMA_twitter_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filepath = '/content/drive/MyDrive/Colab Notebooks/SMA_twitter/'

In [ ]:
import sys
sys.path.append(filepath)

In [ ]:
!pip install tweepy -U

     |████████████████████████████████| 77 kB 3.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import tweepy
import authbeared
import pandas as pd
import requests


In [ ]:
print(tweepy.__version__)
print(requests.__version__)

4.9.0
2.27.1


In [ ]:
def flatten_tweets(tweets_json):
    """ Flattens out tweet dictionaries so relevant JSON
        is in a top-level dictionary."""
    tweets_list = []
    
    # Iterate through each tweet
    for tweet in tweets_json:
        tweet_obj = json.loads(tweet)
    
        # Store the user screen name in 'user-screen_name'
        tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']
    
        # Check if this is a 140+ character tweet
        if 'extended_tweet' in tweet_obj:
            # Store the extended tweet text in 'extended_tweet-full_text'
            tweet_obj['extended_tweet-full_text'] = tweet_obj['extended_tweet']['full_text']
    
        if 'retweeted_status' in tweet_obj:
            # Store the retweet user screen name in 'retweeted_status-user-screen_name'
            tweet_obj['retweeted_status-user-screen_name'] = tweet_obj['retweeted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['retweeted_status-text'] = tweet_obj['retweeted_status']['text']
            
        tweets_list.append(tweet_obj)
    return tweets_list

In [ ]:
client = tweepy.Client(bearer_token=authbeared.bearer_token, return_type = requests.Response,
                        wait_on_rate_limit=True)

In [ ]:
# Replace with your own search query
#query = 'from:suhemparack -is:retweet'  ## -is:retweet : which will not match on Retweets, thus matching only on original Tweets, Quote Tweets, and replies 
                                        ##https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
query = 'intesasanpaolo'   # -is:retweet'
lst_fields = ['author_id','context_annotations','created_at','in_reply_to_user_id','lang','referenced_tweets']
lst_user_fiels = ['username','name','profile_image_url', 'public_metrics']
lst_expansion = ['author_id', 'in_reply_to_user_id','referenced_tweets.id.author_id','referenced_tweets.id']



In [ ]:
#Search tweets
tweets = client.search_recent_tweets(query=query, tweet_fields=lst_fields, user_fields=lst_user_fiels, expansions=lst_expansion, max_results=10) ##https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet
#tweets = client.search_recent_tweets(query=query, tweet_fields=lst_fields, user_fields=['username','name','profile_image_url'], expansions=['author_id'], max_results=10)

# for tweet in tweets.data:
#     print(tweet.text)
#     if len(tweet.context_annotations) > 0:
#         print(tweet.context_annotations)

In [ ]:
tweets

<Response [200]>

In [ ]:
client = tweepy.Client(bearer_token=authbeared.bearer_token)
# #Getting more than 100 Tweets at a time using paginator
# # Replace the limit=1000 with the maximum number of Tweets you want
lst_tweet = []
for tweet in tweepy.Paginator(client.search_recent_tweets, query=query, tweet_fields=lst_fields, user_fields=lst_user_fiels, 
                              expansions=lst_expansion, max_results=10).flatten(limit=20):
    lst_tweet.append(tweet)

# tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
#                               tweet_fields=lst_fields, max_results=10).flatten(limit=10)


In [ ]:
len(lst_tweet)

20

In [ ]:
lst_tweet[0]

<Tweet id=1525004171371585536 text='#IntesaSanpaolo #Albania is focused on the future and delivering its goals, with a strong recovery in 2021 following the challenging financial effects of Covid-19. Read the article to find out more. https://t.co/rLqZ03hsTg https://t.co/zru3FbUqfu'>

In [ ]:
# Save data as dictionary
tweets_dict = tweets.json() 

# Extract "data" value from dictionary
tweets_data = tweets_dict['data'] 
tweets_user =  tweets_dict['includes']['users']
tweets_ref =  tweets_dict['includes']['tweets']

# Transform to pandas Dataframe
df = pd.json_normalize(tweets_data)
df_user = pd.json_normalize(tweets_user)
df_ref = pd.json_normalize(tweets_ref)

In [ ]:
df_ref

,in_reply_to_user_id,lang,text,created_at,author_id,context_annotations,id,referenced_tweets
0,393894382,it,"@intesasanpaolo lancia “Up2Stars”, la challeng...",2022-05-13T06:00:14.000Z,887939702510288896,"[{'domain': {'id': '65', 'name': 'Interests an...",1524992898877603842,NaN
1,NaN,it,🤝Sostieni anche tu con un piccolo contributo i...,2022-05-12T17:06:52.000Z,927420308,NaN,1524798272514232327,NaN
2,NaN,it,Ligabue racconta i suoi inizi ai nostri microf...,2022-05-10T14:38:32.000Z,313204597,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1524036167062151168,NaN
3,NaN,it,🎙️Un viaggio nello spazio con @AstroSamantha: ...,2022-04-30T12:00:01.000Z,393894382,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1520372399774093314,NaN
4,NaN,it,Nuovo accordo tra @Confindustria e @intesasanp...,2022-05-09T15:44:06.000Z,2280596834,NaN,1523690280167383047,NaN
5,NaN,it,L’#ArchivioPublifoto @intesasanpaolo cambia ca...,2022-05-12T18:31:25.000Z,395347215,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524819552567234560,NaN
6,927420308,it,Aiutaci a mantenere intatto lo splendore della...,2022-05-12T17:06:53.000Z,927420308,NaN,1524798275852939266,"[{'type': 'replied_to', 'id': '152479827251423..."


In [ ]:
df

,lang,text,created_at,author_id,context_annotations,id,referenced_tweets,in_reply_to_user_id
0,en,#IntesaSanpaolo #Albania is focused on the fut...,2022-05-13T06:45:02.000Z,566896667,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1525004171371585536,NaN,NaN
1,it,RT @LombardiaInnova: @intesasanpaolo lancia “U...,2022-05-13T06:06:29.000Z,1377704858,"[{'domain': {'id': '65', 'name': 'Interests an...",1524994472316129280,"[{'type': 'retweeted', 'id': '1524992898877603...",NaN
2,it,"@intesasanpaolo lancia “Up2Stars”, la challeng...",2022-05-13T06:00:14.000Z,887939702510288896,"[{'domain': {'id': '65', 'name': 'Interests an...",1524992898877603842,NaN,393894382
3,it,RT @DuomodiMilano: 🤝Sostieni anche tu con un p...,2022-05-13T04:52:07.000Z,2989573967,NaN,1524975754764816404,"[{'type': 'retweeted', 'id': '1524798272514232...",NaN
4,it,RT @RadioItalia: Ligabue racconta i suoi inizi...,2022-05-12T22:03:45.000Z,1006699670,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1524872985085202433,"[{'type': 'retweeted', 'id': '1524036167062151...",NaN
5,it,RT @intesasanpaolo: 🎙️Un viaggio nello spazio ...,2022-05-12T21:22:56.000Z,106724108,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524862715940028431,"[{'type': 'retweeted', 'id': '1520372399774093...",NaN
6,it,RT @Univa_Stampa: Nuovo accordo tra @Confindus...,2022-05-12T21:06:33.000Z,60903266,NaN,1524858592133890050,"[{'type': 'retweeted', 'id': '1523690280167383...",NaN
7,it,RT @gallerieditalia: L’#ArchivioPublifoto @int...,2022-05-12T19:47:29.000Z,393894382,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524838694850646016,"[{'type': 'retweeted', 'id': '1524819552567234...",NaN
8,it,L’#ArchivioPublifoto @intesasanpaolo cambia ca...,2022-05-12T18:31:25.000Z,395347215,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524819552567234560,NaN,NaN
9,it,RT @DuomodiMilano: Aiutaci a mantenere intatto...,2022-05-12T17:25:53.000Z,608125539,NaN,1524803059196502016,"[{'type': 'retweeted', 'id': '1524798275852939...",NaN


In [ ]:
flat_tweets_data = []
for d in tweets_data:
    if 'referenced_tweets' in d:
        print(d['referenced_tweets'])
        for ele in d['referenced_tweets']:
            print(str(ele['type'])+ '_id')
            print(ele['id'])
            d[str(ele['type'])+ '_id'] = ele['id']

    print(d)
    flat_tweets_data.append(d)
df_flat = pd.json_normalize(flat_tweets_data)

{'lang': 'en', 'text': '#IntesaSanpaolo #Albania is focused on the future and delivering its goals, with a strong recovery in 2021 following the challenging financial effects of Covid-19. Read the article to find out more. https://t.co/rLqZ03hsTg https://t.co/zru3FbUqfu', 'created_at': '2022-05-13T06:45:02.000Z', 'author_id': '566896667', 'context_annotations': [{'domain': {'id': '123', 'name': 'Ongoing News Story', 'description': "Ongoing News Stories like 'Brexit'"}, 'entity': {'id': '1220701888179359745', 'name': 'COVID-19'}}], 'id': '1525004171371585536'}
[{'type': 'retweeted', 'id': '1524992898877603842'}]
retweeted_id
1524992898877603842
{'lang': 'it', 'text': 'RT @LombardiaInnova: @intesasanpaolo lancia “Up2Stars”, la challenge per stimolare il potenziale di #innovazione delle #startup che desider…', 'referenced_tweets': [{'type': 'retweeted', 'id': '1524992898877603842'}], 'created_at': '2022-05-13T06:06:29.000Z', 'author_id': '1377704858', 'context_annotations': [{'domain': {'

In [ ]:
df_flat

,lang,text,created_at,author_id,context_annotations,id,referenced_tweets,retweeted_id,in_reply_to_user_id
0,en,#IntesaSanpaolo #Albania is focused on the fut...,2022-05-13T06:45:02.000Z,566896667,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1525004171371585536,NaN,NaN,NaN
1,it,RT @LombardiaInnova: @intesasanpaolo lancia “U...,2022-05-13T06:06:29.000Z,1377704858,"[{'domain': {'id': '65', 'name': 'Interests an...",1524994472316129280,"[{'type': 'retweeted', 'id': '1524992898877603...",1524992898877603842,NaN
2,it,"@intesasanpaolo lancia “Up2Stars”, la challeng...",2022-05-13T06:00:14.000Z,887939702510288896,"[{'domain': {'id': '65', 'name': 'Interests an...",1524992898877603842,NaN,NaN,393894382
3,it,RT @DuomodiMilano: 🤝Sostieni anche tu con un p...,2022-05-13T04:52:07.000Z,2989573967,NaN,1524975754764816404,"[{'type': 'retweeted', 'id': '1524798272514232...",1524798272514232327,NaN
4,it,RT @RadioItalia: Ligabue racconta i suoi inizi...,2022-05-12T22:03:45.000Z,1006699670,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1524872985085202433,"[{'type': 'retweeted', 'id': '1524036167062151...",1524036167062151168,NaN
5,it,RT @intesasanpaolo: 🎙️Un viaggio nello spazio ...,2022-05-12T21:22:56.000Z,106724108,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524862715940028431,"[{'type': 'retweeted', 'id': '1520372399774093...",1520372399774093314,NaN
6,it,RT @Univa_Stampa: Nuovo accordo tra @Confindus...,2022-05-12T21:06:33.000Z,60903266,NaN,1524858592133890050,"[{'type': 'retweeted', 'id': '1523690280167383...",1523690280167383047,NaN
7,it,RT @gallerieditalia: L’#ArchivioPublifoto @int...,2022-05-12T19:47:29.000Z,393894382,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524838694850646016,"[{'type': 'retweeted', 'id': '1524819552567234...",1524819552567234560,NaN
8,it,L’#ArchivioPublifoto @intesasanpaolo cambia ca...,2022-05-12T18:31:25.000Z,395347215,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524819552567234560,NaN,NaN,NaN
9,it,RT @DuomodiMilano: Aiutaci a mantenere intatto...,2022-05-12T17:25:53.000Z,608125539,NaN,1524803059196502016,"[{'type': 'retweeted', 'id': '1524798275852939...",1524798275852939266,NaN


In [ ]:
df_ref

,in_reply_to_user_id,lang,text,created_at,author_id,context_annotations,id,referenced_tweets
0,393894382,it,"@intesasanpaolo lancia “Up2Stars”, la challeng...",2022-05-13T06:00:14.000Z,887939702510288896,"[{'domain': {'id': '65', 'name': 'Interests an...",1524992898877603842,NaN
1,NaN,it,🤝Sostieni anche tu con un piccolo contributo i...,2022-05-12T17:06:52.000Z,927420308,NaN,1524798272514232327,NaN
2,NaN,it,Ligabue racconta i suoi inizi ai nostri microf...,2022-05-10T14:38:32.000Z,313204597,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1524036167062151168,NaN
3,NaN,it,🎙️Un viaggio nello spazio con @AstroSamantha: ...,2022-04-30T12:00:01.000Z,393894382,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1520372399774093314,NaN
4,NaN,it,Nuovo accordo tra @Confindustria e @intesasanp...,2022-05-09T15:44:06.000Z,2280596834,NaN,1523690280167383047,NaN
5,NaN,it,L’#ArchivioPublifoto @intesasanpaolo cambia ca...,2022-05-12T18:31:25.000Z,395347215,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524819552567234560,NaN
6,927420308,it,Aiutaci a mantenere intatto lo splendore della...,2022-05-12T17:06:53.000Z,927420308,NaN,1524798275852939266,"[{'type': 'replied_to', 'id': '152479827251423..."


In [ ]:
df_user

,username,name,id,profile_image_url,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,TelliniMassim,Max Tellini,566896667,https://pbs.twimg.com/profile_images/133897564...,1165,1581,16568,110
1,startzai,Startzai,1377704858,https://pbs.twimg.com/profile_images/363588122...,4351,302,197860,3329
2,LombardiaInnova,Open Innovation,887939702510288896,https://pbs.twimg.com/profile_images/887941498...,2903,821,5503,37
3,intesasanpaolo,Intesa Sanpaolo,393894382,https://pbs.twimg.com/profile_images/875695525...,16695,762,12340,212
4,DanielaAzzolini,Daniela Azzolini,2989573967,https://abs.twimg.com/sticky/default_profile_i...,7,24,204,1
5,DuomodiMilano,Duomo di Milano,927420308,https://pbs.twimg.com/profile_images/687595658...,14191,1469,12436,217
6,topo1966,roberto bertorelle,1006699670,https://pbs.twimg.com/profile_images/136095082...,723,4996,38709,9
7,RadioItalia,Radio Italia,313204597,https://pbs.twimg.com/profile_images/150366123...,719407,797,55947,731
8,FedeLupoli,Federica Lupoli,106724108,https://pbs.twimg.com/profile_images/145597506...,373,375,1269,6
9,Cionfry,Davide Cionfrini,60903266,https://pbs.twimg.com/profile_images/799391506...,698,679,2826,27


In [ ]:
df

,lang,text,created_at,author_id,context_annotations,id,referenced_tweets,in_reply_to_user_id
0,en,#IntesaSanpaolo #Albania is focused on the fut...,2022-05-13T06:45:02.000Z,566896667,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1525004171371585536,NaN,NaN
1,it,RT @LombardiaInnova: @intesasanpaolo lancia “U...,2022-05-13T06:06:29.000Z,1377704858,"[{'domain': {'id': '65', 'name': 'Interests an...",1524994472316129280,"[{'type': 'retweeted', 'id': '1524992898877603...",NaN
2,it,"@intesasanpaolo lancia “Up2Stars”, la challeng...",2022-05-13T06:00:14.000Z,887939702510288896,"[{'domain': {'id': '65', 'name': 'Interests an...",1524992898877603842,NaN,393894382
3,it,RT @DuomodiMilano: 🤝Sostieni anche tu con un p...,2022-05-13T04:52:07.000Z,2989573967,NaN,1524975754764816404,"[{'type': 'retweeted', 'id': '1524798272514232...",NaN
4,it,RT @RadioItalia: Ligabue racconta i suoi inizi...,2022-05-12T22:03:45.000Z,1006699670,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1524872985085202433,"[{'type': 'retweeted', 'id': '1524036167062151...",NaN
5,it,RT @intesasanpaolo: 🎙️Un viaggio nello spazio ...,2022-05-12T21:22:56.000Z,106724108,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524862715940028431,"[{'type': 'retweeted', 'id': '1520372399774093...",NaN
6,it,RT @Univa_Stampa: Nuovo accordo tra @Confindus...,2022-05-12T21:06:33.000Z,60903266,NaN,1524858592133890050,"[{'type': 'retweeted', 'id': '1523690280167383...",NaN
7,it,RT @gallerieditalia: L’#ArchivioPublifoto @int...,2022-05-12T19:47:29.000Z,393894382,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524838694850646016,"[{'type': 'retweeted', 'id': '1524819552567234...",NaN
8,it,L’#ArchivioPublifoto @intesasanpaolo cambia ca...,2022-05-12T18:31:25.000Z,395347215,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524819552567234560,NaN,NaN
9,it,RT @DuomodiMilano: Aiutaci a mantenere intatto...,2022-05-12T17:25:53.000Z,608125539,NaN,1524803059196502016,"[{'type': 'retweeted', 'id': '1524798275852939...",NaN


In [ ]:
df['referenced_tweets'][9]

[{'id': '1524798275852939266', 'type': 'retweeted'}]

In [ ]:
586429795

586429795

In [ ]:
df['context_annotations'][3]

nan

In [ ]:
df_user

,username,name,id,profile_image_url,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,TelliniMassim,Max Tellini,566896667,https://pbs.twimg.com/profile_images/133897564...,1165,1581,16568,110
1,startzai,Startzai,1377704858,https://pbs.twimg.com/profile_images/363588122...,4351,302,197860,3329
2,LombardiaInnova,Open Innovation,887939702510288896,https://pbs.twimg.com/profile_images/887941498...,2903,821,5503,37
3,intesasanpaolo,Intesa Sanpaolo,393894382,https://pbs.twimg.com/profile_images/875695525...,16695,762,12340,212
4,DanielaAzzolini,Daniela Azzolini,2989573967,https://abs.twimg.com/sticky/default_profile_i...,7,24,204,1
5,DuomodiMilano,Duomo di Milano,927420308,https://pbs.twimg.com/profile_images/687595658...,14191,1469,12436,217
6,topo1966,roberto bertorelle,1006699670,https://pbs.twimg.com/profile_images/136095082...,723,4996,38709,9
7,RadioItalia,Radio Italia,313204597,https://pbs.twimg.com/profile_images/150366123...,719407,797,55947,731
8,FedeLupoli,Federica Lupoli,106724108,https://pbs.twimg.com/profile_images/145597506...,373,375,1269,6
9,Cionfry,Davide Cionfrini,60903266,https://pbs.twimg.com/profile_images/799391506...,698,679,2826,27


In [ ]:
1947105817

1947105817

In [ ]:
df_user

,username,name,id,profile_image_url,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,TelliniMassim,Max Tellini,566896667,https://pbs.twimg.com/profile_images/133897564...,1165,1581,16568,110
1,startzai,Startzai,1377704858,https://pbs.twimg.com/profile_images/363588122...,4351,302,197860,3329
2,LombardiaInnova,Open Innovation,887939702510288896,https://pbs.twimg.com/profile_images/887941498...,2903,821,5503,37
3,intesasanpaolo,Intesa Sanpaolo,393894382,https://pbs.twimg.com/profile_images/875695525...,16695,762,12340,212
4,DanielaAzzolini,Daniela Azzolini,2989573967,https://abs.twimg.com/sticky/default_profile_i...,7,24,204,1
5,DuomodiMilano,Duomo di Milano,927420308,https://pbs.twimg.com/profile_images/687595658...,14191,1469,12436,217
6,topo1966,roberto bertorelle,1006699670,https://pbs.twimg.com/profile_images/136095082...,723,4996,38709,9
7,RadioItalia,Radio Italia,313204597,https://pbs.twimg.com/profile_images/150366123...,719407,797,55947,731
8,FedeLupoli,Federica Lupoli,106724108,https://pbs.twimg.com/profile_images/145597506...,373,375,1269,6
9,Cionfry,Davide Cionfrini,60903266,https://pbs.twimg.com/profile_images/799391506...,698,679,2826,27


In [ ]:
tweets_dict['includes']

{'tweets': [{'author_id': '887939702510288896',
   'context_annotations': [{'domain': {'description': 'Top level interests and hobbies groupings, like Food or Travel',
      'id': '65',
      'name': 'Interests and Hobbies Vertical'},
     'entity': {'id': '781974596148793345', 'name': 'Business & finance'}},
    {'domain': {'description': 'A grouping of interests and hobbies entities, like Novelty Food or Destinations',
      'id': '66',
      'name': 'Interests and Hobbies Category'},
     'entity': {'description': 'Startups',
      'id': '849075881653846016',
      'name': 'Startups'}}],
   'created_at': '2022-05-13T06:00:14.000Z',
   'id': '1524992898877603842',
   'in_reply_to_user_id': '393894382',
   'lang': 'it',
   'text': '@intesasanpaolo lancia “Up2Stars”, la challenge per stimolare il potenziale di #innovazione delle #startup che desiderano crescere sotto il profilo manageriale e finanziario.\nPer saperne di più 👉https://t.co/sMWSQX1xjZ https://t.co/mywmnjra7Z'},
  {'author

In [ ]:
df

,lang,text,created_at,author_id,context_annotations,id,referenced_tweets,in_reply_to_user_id
0,en,#IntesaSanpaolo #Albania is focused on the fut...,2022-05-13T06:45:02.000Z,566896667,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1525004171371585536,NaN,NaN
1,it,RT @LombardiaInnova: @intesasanpaolo lancia “U...,2022-05-13T06:06:29.000Z,1377704858,"[{'domain': {'id': '65', 'name': 'Interests an...",1524994472316129280,"[{'type': 'retweeted', 'id': '1524992898877603...",NaN
2,it,"@intesasanpaolo lancia “Up2Stars”, la challeng...",2022-05-13T06:00:14.000Z,887939702510288896,"[{'domain': {'id': '65', 'name': 'Interests an...",1524992898877603842,NaN,393894382
3,it,RT @DuomodiMilano: 🤝Sostieni anche tu con un p...,2022-05-13T04:52:07.000Z,2989573967,NaN,1524975754764816404,"[{'type': 'retweeted', 'id': '1524798272514232...",NaN
4,it,RT @RadioItalia: Ligabue racconta i suoi inizi...,2022-05-12T22:03:45.000Z,1006699670,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1524872985085202433,"[{'type': 'retweeted', 'id': '1524036167062151...",NaN
5,it,RT @intesasanpaolo: 🎙️Un viaggio nello spazio ...,2022-05-12T21:22:56.000Z,106724108,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524862715940028431,"[{'type': 'retweeted', 'id': '1520372399774093...",NaN
6,it,RT @Univa_Stampa: Nuovo accordo tra @Confindus...,2022-05-12T21:06:33.000Z,60903266,NaN,1524858592133890050,"[{'type': 'retweeted', 'id': '1523690280167383...",NaN
7,it,RT @gallerieditalia: L’#ArchivioPublifoto @int...,2022-05-12T19:47:29.000Z,393894382,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524838694850646016,"[{'type': 'retweeted', 'id': '1524819552567234...",NaN
8,it,L’#ArchivioPublifoto @intesasanpaolo cambia ca...,2022-05-12T18:31:25.000Z,395347215,"[{'domain': {'id': '29', 'name': 'Events [Enti...",1524819552567234560,NaN,NaN
9,it,RT @DuomodiMilano: Aiutaci a mantenere intatto...,2022-05-12T17:25:53.000Z,608125539,NaN,1524803059196502016,"[{'type': 'retweeted', 'id': '1524798275852939...",NaN


In [ ]:
df.loc[2,'context_annotations']

[{'domain': {'description': 'Top level interests and hobbies groupings, like Food or Travel',
   'id': '65',
   'name': 'Interests and Hobbies Vertical'},
  'entity': {'id': '781974596148793345', 'name': 'Business & finance'}},
 {'domain': {'description': 'A grouping of interests and hobbies entities, like Novelty Food or Destinations',
   'id': '66',
   'name': 'Interests and Hobbies Category'},
  'entity': {'description': 'Startups',
   'id': '849075881653846016',
   'name': 'Startups'}}]

# Retweet

In [ ]:
#client.get_retweeters(id, *, expansions=None, max_results=None, media_fields=None, pagination_token=None, place_fields=None, poll_fields=None, tweet_fields=None, user_fields=None, user_auth=False)
client.get_retweeters(1524611911911886849)

Response(data=None, includes={}, errors=[], meta={'result_count': 0})

# Twarc2

In [ ]:
import json 

In [ ]:
json.load(file object)

In [ ]:
import json
  
# Opening JSON file
f = open(filepath+'results_1.jsonl')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

JSONDecodeError: ignored

In [ ]:
data.keys()

dict_keys(['data', 'includes', 'meta', '__twarc'])

In [ ]:
df_includes = pd.json_normalize(data['includes']['tweets'])
df_includes

In [ ]:
df_ref = pd.json_normalize(data['data'])
df_ref.iloc[0]

source                                                             Hootsuite Inc.
reply_settings                                                           everyone
referenced_tweets               [{'type': 'retweeted', 'id': '1525016456731185...
context_annotations             [{'domain': {'id': '65', 'name': 'Interests an...
text                            RT @valentina_magri: #fondicomuni, chi li comp...
id                                                            1525029769447976960
conversation_id                                               1525029769447976960
lang                                                                           it
possibly_sensitive                                                          False
created_at                                               2022-05-13T08:26:45.000Z
author_id                                                               159211191
entities.hashtags               [{'start': 21, 'end': 33, 'tag': 'fondicomuni'...
entities.mention

In [ ]:
df_ref['referenced_tweets'].iloc[0]

[{'id': '1525016456731185152', 'type': 'retweeted'}]

In [ ]:
df_ref['created_at'].iloc[0]

'2022-05-13T08:26:45.000Z'

In [ ]:
df_ref['created_at'].iloc[99]

'2022-05-11T15:58:22.000Z'

In [ ]:
min(df_ref.created_at)

'2022-05-11T15:58:22.000Z'

In [ ]:
max(df_ref.created_at)

'2022-05-13T08:26:45.000Z'

In [ ]:
len(df_ref.text.unique())

83

In [ ]:
lst_ref = df_ref[~df_ref.referenced_tweets.isnull()].referenced_tweets

In [ ]:
# Opening JSON file
f = open(filepath+'flatten.jsonl')
  
# returns JSON object as 
# a dictionary
flatten = json.load(f)



JSONDecodeError: ignored

In [ ]:
filepath

NameError: ignored

In [ ]:
!ls $filepath

sample_data


In [ ]:
flatten = [json.loads(line) for line in open(filepath+'results_1.jsonl', 'r')]

In [ ]:
flatten[0]['data']

In [ ]:
df_flatten_0 = pd.json_normalize(flatten[0]['data'])
df_flatten_1 = pd.json_normalize(flatten[1]['data'])

In [ ]:
dtf_flatten = pd.DataFrame()
for dct_f in flatten:
  dct_data = []
  for d in dct_f['data']:
      if 'referenced_tweets' in d:
          # print(d['referenced_tweets'])
          for ele in d['referenced_tweets']:
              # print(str(ele['type'])+ '_id')
              # print(ele['id'])
              d[str(ele['type'])+ '_id'] = ele['id']

      # print(d)
      dct_data.append(d)
  #flatten referenced_tweets
  dtf_flatten = pd.concat([dtf_flatten,
                           pd.json_normalize(dct_data)])
  
print(dtf_flatten.shape)
print(dtf_flatten.columns)
dtf_flatten.head(10)

(200, 26)
Index(['lang', 'text', 'conversation_id', 'created_at', 'author_id', 'source',
       'context_annotations', 'id', 'possibly_sensitive', 'reply_settings',
       'entities.hashtags', 'entities.urls', 'entities.mentions',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count',
       'referenced_tweets', 'retweeted_id', 'attachments.media_keys',
       'in_reply_to_user_id', 'replied_to_id', 'entities.annotations',
       'entities.cashtags', 'quoted_id', 'geo.place_id'],
      dtype='object')


,lang,text,conversation_id,created_at,author_id,source,context_annotations,id,possibly_sensitive,reply_settings,...,public_metrics.quote_count,referenced_tweets,retweeted_id,attachments.media_keys,in_reply_to_user_id,replied_to_id,entities.annotations,entities.cashtags,quoted_id,geo.place_id
0,it,wallstreetita: RT @valentina_magri: #fondicomu...,1525041287241650177,2022-05-13T09:12:31.000Z,2754489039,IFTTT,"[{'domain': {'id': '65', 'name': 'Interests an...",1525041287241650177,False,everyone,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,it,RT @ItalyinFrance: Si apre oggi in Ambasciata ...,1525041264747548673,2022-05-13T09:12:26.000Z,27883774,Twitter for Android,NaN,1525041264747548673,False,everyone,...,0,"[{'type': 'retweeted', 'id': '1525040622792589...",1525040622792589312,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,it,Si apre oggi in Ambasciata a Parigi il terzo F...,1525040622792589312,2022-05-13T09:09:53.000Z,968165303766134786,Twitter for iPhone,NaN,1525040622792589312,False,everyone,...,0,NaN,NaN,"[3_1525040606640209922, 3_1525040606908760065,...",NaN,NaN,NaN,NaN,NaN,NaN
3,it,@AdolfoRossi1 @AgenziaCoesione @fsitaliane @in...,1524998421479931904,2022-05-13T09:08:33.000Z,381532613,Twitter Web App,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1525040290637266945,False,everyone,...,0,"[{'type': 'replied_to', 'id': '152502150926956...",NaN,NaN,438353555,1525021509269561345,NaN,NaN,NaN,NaN
4,it,📈 Su #FlashAzioni seduta volatile per Wall Str...,1525039591958388738,2022-05-13T09:05:47.000Z,393894382,Twitter Web App,NaN,1525039591958388738,False,everyone,...,0,NaN,NaN,[3_1525039587151732736],NaN,NaN,NaN,NaN,NaN,NaN
5,en,Intesa Sanpaolo Albania is focused on the futu...,1525033065617170433,2022-05-13T08:39:51.000Z,3014618632,Hootsuite Inc.,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1525033065617170433,False,everyone,...,0,NaN,NaN,[3_1525033063784361984],NaN,NaN,"[{'start': 0, 'end': 22, 'probability': 0.6952...",NaN,NaN,NaN
6,it,"Fondo per l’Ambiente Italiano ETS, @Fondoambie...",1525032241780359168,2022-05-13T08:36:34.000Z,2997335747,Twitter Web App,NaN,1525032241780359168,False,everyone,...,0,NaN,NaN,[3_1525031858416885761],NaN,NaN,NaN,NaN,NaN,NaN
7,it,🤝A Como è stato presentato il nuovo Accordo tr...,1525031864313978882,2022-05-13T08:35:04.000Z,2545514396,Twitter Web App,NaN,1525031864313978882,False,everyone,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,it,"RT @valentina_magri: #fondicomuni, chi li comp...",1525029769447976960,2022-05-13T08:26:45.000Z,159211191,Hootsuite Inc.,"[{'domain': {'id': '65', 'name': 'Interests an...",1525029769447976960,False,everyone,...,0,"[{'type': 'retweeted', 'id': '1525016456731185...",1525016456731185152,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,en,🟢 #Investors #Financial The consolidated Inter...,1525029085801594880,2022-05-13T08:24:02.000Z,393894382,Twitter Web App,"[{'domain': {'id': '65', 'name': 'Interests an...",1525029085801594880,False,everyone,...,0,NaN,NaN,[3_1525029010056658946],NaN,NaN,"[{'start': 67, 'end': 87, 'probability': 0.361...",NaN,NaN,NaN


In [ ]:
len(dtf_flatten.text.unique())

154

In [ ]:
dtf_flatten[~dtf_flatten.retweeted_id.isnull()].shape

(106, 26)

In [ ]:
set(df_flatten_1.columns) - set(df_flatten_0.columns)

{'geo.place_id'}

In [ ]:
df_flatten.columns

Index(['referenced_tweets', 'possibly_sensitive', 'lang', 'created_at', 'text',
       'source', 'id', 'conversation_id', 'reply_settings', 'author_id',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count',
       'entities.hashtags', 'entities.mentions', 'context_annotations',
       'entities.urls', 'attachments.media_keys', 'in_reply_to_user_id',
       'entities.annotations', 'entities.cashtags', 'geo.place_id'],
      dtype='object')